In [1]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [13]:
#---------------------------------------
# EDA
#---------------------------------------
# 데이터 샘플
# train.head()
# train.tail(n)
# train.sample(n)
# # 데이터 크기
# train.shape, test.shape
# # 자료형 (타입)
# train.info()
# # 수치형 컬럼 통계값 확인(train)
# train.describe()
# # 수치형 컬럼 통계값 확인(test)
# test.describe()
# # 범주형 컬럼 통계값 확인 (train)
# train.describe(include='O')
# # 범주형 컬럼 통계값 확인 (test)
# test.describe(include='object')
# # 결측치 (train)
# train.isnull().sum()
# # 결측치 (test)
# test.isnull().sum()
# # label(target)별 개수 확인
# train['Exited'].value_counts()

In [ ]:
#---------------
#데이터 전처리
#----------------
#결측치 처리
# 이상치

# train.descibe()


In [2]:
y_train = train.pop('Exited')

In [3]:
train.shape , test.shape

((165034, 13), (110023, 13))

In [9]:
#레이블 인코딩

In [14]:
from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object]
for col in cols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col],test[col]],axis=0))# 행 합치기 train,test 행이 맞지 않음
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

test

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,0,0,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,0,0,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,0,0,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,0,1,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,1,1,38.0,10,121263.62,1,1.0,0.0,139431.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110018,275052,15662091,P'eng,570,2,1,29.0,7,116099.82,1,1.0,1.0,148087.62
110019,275053,15774133,Cox,575,0,0,36.0,4,178032.53,1,1.0,1.0,42181.68
110020,275054,15728456,Ch'iu,712,0,1,31.0,2,0.00,2,1.0,0.0,16287.38
110021,275055,15687541,Yegorova,709,0,0,32.0,3,0.00,1,1.0,1.0,158816.58


In [16]:
y_train.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [ ]:
# 검증용 데이터 분할

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((132027, 13), (33007, 13), (132027,), (33007,))

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred1=rf.predict_proba(X_val)  # auc용
pred1_label=rf.predict(X_val) # auc제외 나머지 검증용 라벨 ,정확도

# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred2 = lgbmc.predict_proba(X_val)
pred2_label=rf.predict(X_val)

In [21]:
print(pred1[:,1])
print()
print(pred2[:,1])

[0.11 0.21 0.75 ... 0.28 0.05 0.36]

[0.06442807 0.14527317 0.68723293 ... 0.23911734 0.02022277 0.30957594]


In [28]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred1[:,1])
print('roc_auc:', roc_auc)

from sklearn.metrics import accuracy_score
# pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, pred1_label)
print('accuracy_score:', accuracy)

from sklearn.metrics import f1_score
f1 = f1_score(y_val,  pred1_label, pos_label=1) 
print('f1_score:', f1)

roc_auc: 0.8801525772952798
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [31]:
from sklearn.metrics import roc_auc_score
roc_auc2_lgm = roc_auc_score(y_val, pred2[:,1])
print('roc_auc:', roc_auc2_lgm)

from sklearn.metrics import accuracy_score
# pred = rf.predict(X_val)
accuracy_lgm = accuracy_score(y_val, pred2_label)
print('accuracy_score:', accuracy_lgm)

from sklearn.metrics import f1_score
f1_lgm = f1_score(y_val,  pred2_label, pos_label=1)
print('f1_score:', f1_lgm)

roc_auc: 0.8902162094130004
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [34]:
# pred = lgbmc.predict_proba(test)
# pred

# # lgbmc.classes_

# submit = pd.DataFrame({'pred':pred[:,1]})
# submit.to_csv("result.csv", index=False)

# pd.read_csv("result.csv")